# Convolutional neural network with PyTorch

In [1]:
# Imports
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# Load data
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [80]:
# Convert data into numpy arrays then into tensors
X_train = torch.tensor(train.iloc[:, 1:].values, dtype=torch.float32)
X_test = torch.tensor(test.values, dtype=torch.float32)
y_train_original = torch.tensor(train.iloc[:, 0].values, dtype=torch.long)

In [81]:
# One-hot encode y_train
num_classes = 10  
y_train = F.one_hot(y_train_original, num_classes=num_classes)

In [82]:
# Convert y_train to float 32
y_train = y_train.float()
y_train[0]

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [83]:
# Reshape tensors for convnet
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [84]:
X_train.shape

torch.Size([42000, 28, 28, 1])

In [85]:
# Create data loader
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [43]:
# Define CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [57]:
# Initialize model
model = CNN()

In [58]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [66]:
# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        # Reshape data from [batch_size, 28, 28, 1] to [batch_size, 1, 28, 28]
        data = data.permute(0, 3, 1, 2)
        data, target = data.to(device), target.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx % 100 == 99:    # print every 100 mini-batches
            print(f'[{epoch + 1}, {batch_idx + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 1.898
[1,   200] loss: 0.162
[1,   300] loss: 0.125
[1,   400] loss: 0.100
[1,   500] loss: 0.099
[1,   600] loss: 0.097
[2,   100] loss: 0.068
[2,   200] loss: 0.069
[2,   300] loss: 0.066
[2,   400] loss: 0.068
[2,   500] loss: 0.057
[2,   600] loss: 0.067
[3,   100] loss: 0.046
[3,   200] loss: 0.041
[3,   300] loss: 0.049
[3,   400] loss: 0.036
[3,   500] loss: 0.053
[3,   600] loss: 0.056
[4,   100] loss: 0.033
[4,   200] loss: 0.039
[4,   300] loss: 0.024
[4,   400] loss: 0.042
[4,   500] loss: 0.032
[4,   600] loss: 0.044
[5,   100] loss: 0.021
[5,   200] loss: 0.014
[5,   300] loss: 0.036
[5,   400] loss: 0.027
[5,   500] loss: 0.042
[5,   600] loss: 0.038
[6,   100] loss: 0.029
[6,   200] loss: 0.022
[6,   300] loss: 0.028
[6,   400] loss: 0.028
[6,   500] loss: 0.030
[6,   600] loss: 0.039
[7,   100] loss: 0.023
[7,   200] loss: 0.027
[7,   300] loss: 0.032
[7,   400] loss: 0.021
[7,   500] loss: 0.027
[7,   600] loss: 0.025
[8,   100] loss: 0.017
[8,   200] 

In [87]:
# Evaluate the model
model.eval()

# New data loader with original y_train
eval_dataset = TensorDataset(X_train, y_train_original)
eval_dataloader = DataLoader(eval_dataset, batch_size=64, shuffle=True)

In [89]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in eval_dataloader:
        images = images.permute(0, 3, 1, 2)
        outputs = model(images)
        _, predictions = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()

    print(f'Accuracy of the model on the training images: {100 * correct / total}%')

Accuracy of the model on the training images: 99.17619047619047%


In [70]:
X_test = X_test.to(device)
X_test = X_test.permute(0, 3, 1, 2)
# Make predictions
model.eval()

with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)

In [74]:
# Submission
submission = pd.DataFrame({
    "ImageId": range(1, 28001),
    "Label": predicted
})
submission.to_csv("Submission.csv", index=False)